In [19]:
import pandas as pd
import string

In [20]:
# Data Preparation
with open('slips-tag.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [21]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [22]:
# Load Data and convert into Pandas DataFrame
df = pd.DataFrame(data[1:],columns=data[0])

In [ ]:
df.head()

In [24]:
# See whitespaces
# string.whitespace
# string.punctuation

whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [25]:
df['text'] = df['text'].apply(cleanText)

In [ ]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

In [ ]:
dataClean.head(10)

In [28]:
# Convert Data into spacy format
group = dataClean.groupby(by='id')

In [30]:
grouparray = group.get_group('Slip.png')[['text','tag']].values
content = ''
annotations = {'entities':[]}
start = 0
end = 0
for text, label in grouparray:
    text = str(text)
    stringLenght = len(text) + 1

    start = end
    end = start + stringLenght

    if label != 'O':
        annot = (start,end -1 ,label)
        annotations['entities'].append(annot)

    content = content + text + ' '

In [ ]:
content

In [32]:
slips = group.groups.keys()

In [34]:
allSlips = []
for slip in slips:
    slipData = []
    grouparray = group.get_group(slip)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    slipData = (content,annotations)
    allSlips.append(slipData)

In [ ]:
allSlips

In [36]:
# Spliting Data into Training and Testing Set
import random

random.shuffle(allSlips)
len(allSlips)

1

In [37]:
TrainData = allSlips[:240]
TestData = allSlips[240:]

In [38]:
# Save data
import pickle

pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))

# GoTo: https://spacy.io/usage/training and download the config.cfg file for NL training models